In [1]:
import pandas as pd
import geopandas as gpd
from datetime import date, datetime, timedelta
import json
from shapely.geometry import box, mapping
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata
from functools import partial
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import pandas.io.sql as psql
import os
import xarray as xr



This code is loading environment variables from a .env and creating a connection to a database using the POSTGRES_DATABASE, POSTGRES_USER and POSTGRES_PASSWORD environment variables.

It then sets some default time variables for reference. This is followed by setting time query range between modifiable min_date and max_date.

It then fetches required data from Postgresql engine as pandas dataframe. The code then sets dtypes of certain fields like 'mile', 'lat_g' and 'lon_g' to simplify operations. At the same time, it also converts the time field to datetime in order to build a list of dates to iterate the WSE interpolation from. It uses function format_time() to convert the POSIX time to datetime string.

In [2]:
##Loading environment variables
'''
dotenv_path = Path('.env') # Load environment file
load_dotenv(dotenv_path=dotenv_path) # Loads environment variables from .env file inside the project
'''
load_dotenv() # Loads environment variables from a .env file in the current working directory

## Defining variables to connect to it
dbname = os.environ.get("POSTGRES_DATABASE") # Defines database name variable by retrieving value from environment variable
user_db = os.environ.get("POSTGRES_USER") # Defines user variable by retrieving value from environment variable
password = os.environ.get("POSTGRES_PASSWORD") # Defines password variable by retrieving value from environment variable
host="10.3.10.19" # Database host or IP address as string
port="5432" # Port number of database as string

## Create connection
engine = create_engine(f"postgresql://{user_db}:{password}@{host}:{port}/{dbname}") # Creates an engine connection with PostgreSQL using the variables above

## Set some default time variables for reference
today = date.today() # Gets today's date as date object
yesterday = today - timedelta(days = 1) # Gets yesterday's date as date object
now = datetime.now() # Gets current date and time as datetime object

##Set time query
min_date = '2021-05-24 00:00:00' # Modifiable - Sets minimum date for query
min_date = pd.Timestamp(min_date).strftime('%Y-%m-%d') # Converts min_date from datetime object to a string
max_date = '2023-01-01 00:00:00' # Modifiable - Sets maximum date for query
max_date = pd.Timestamp(max_date).strftime('%Y-%m-%d') # Converts max_date from datetime object to a string

## Retrieves data from rg_wse_3h_subset between min_date and max_date 
sql = f"SELECT * FROM rg_wse_3h_subset WHERE time<'{max_date}'and time>'{min_date}'" # and sid='rg_01160'" # Optional condition to filter data further
wse_df = psql.read_sql(sql, engine) # Executes SQL query and stores result as dataframe
wse_df_types = wse_df.dtypes # Gets data types of columns in dataframe

##Use the time field to build a list to iterate the wse interpolation from
wse_df = wse_df.sort_values('time') # Sorts values in dataframe by time column
wse_period_range = wse_df['time'].drop_duplicates().values.tolist() # Generates list of unique time values from dataframe

##Convert POSIX time to datetime then string
def format_time(time): # Defines function to convert POSIX time to datetime/string format
    format = pd.to_datetime(time) # Converts POSIX time to datetime type
    return format.strftime('%Y-%m-%d %H:%M:%S') # Converts datetime type to string

wse_period_list = list((map(lambda t: format_time(t), wse_period_range))) # Generates list of strings from datetime objects
resolution = r = 500 # Assign default resolution for following interpolation
directory = 'Z:\\wse\\' # Set directory location for output WSE surfaces
directory_str = directory + f'wse_{r}m_' # Set directory path string for combined daily netcdf output

In [10]:
start = pd.Timestamp(min_date) + timedelta(days = 1)
end = pd.Timestamp(max_date) - timedelta(days = 1)
# Find date range of days in the given range 
day_list = []
day_range = pd.date_range(start=start, end=end, freq="D")

for d in day_range:
    day_list.append('y' + d.strftime('%Y') + '_d' + d.strftime('%j'))

# Create period_dict to store the output data 
period_dict = {}

# Loop over the day_range
for d in day_range:
 
    # Set previous step to 3 hour earlier from the d
    previous_step = d - timedelta(hours=3)
    
    # Set next step to 1 day from the d
    next_step = d + timedelta(days=1)
    
    # Initialize dictionary with string version of 'd' as key
    period_dict[d.strftime('y%Y_d%j')] = []
    
    # Find date range of the hours in between previous_step and next_step with the frequency of 3 hours
    range = pd.date_range(start=previous_step, end=next_step, freq='3H')

    # Loop over the range 
    for t in range:
        # Make filename template by filling in the required data
        stamp = t.strftime('%Y%m%dT%H%M')
        day = t.strftime('%j')
        #filename_template = "{directory_str}{stamp}_y{t.year}_d{t.day_of_year}.nc"
        filename_template = "{directory_str}{stamp}_y{t.year}_d{day}.nc"
        
        # Execute format on the filename_template 
        ind_period = filename_template.format(directory_str=directory_str, stamp=stamp, t=t, day=day)
        
        # Store the ind_period in the period_dict with the selected key
        period_dict[d.strftime('y%Y_d%j')].append(ind_period)



In [11]:
period_dict

{'y2021_d145': ['Z:\\wse\\wse_500m_20210524T2100_y2021_d144.nc',
  'Z:\\wse\\wse_500m_20210525T0000_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T0300_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T0600_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T0900_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T1200_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T1500_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T1800_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210525T2100_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210526T0000_y2021_d146.nc'],
 'y2021_d146': ['Z:\\wse\\wse_500m_20210525T2100_y2021_d145.nc',
  'Z:\\wse\\wse_500m_20210526T0000_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T0300_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T0600_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T0900_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T1200_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T1500_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T1800_y2021_d146.nc',
  'Z:\\wse\\wse_500m_20210526T2100_y2021_d146.nc',
  

This code defines the wse_interp function which is used to interpolate missing Water Surface Elevations (WSE) for a given period of time. With time series gauge data stored in a pandas dataframe ('wse_df') as input, the script outputs WSEs converted to a geo-referenced format in netCDF. The process involves interpolating missing values between known elevation readings for a given time period, rasterizing the result using gridding formulas, and exporting the results with naming conventions that identify the selection and timeframe used. 

It begins by passing the period selection to create a subset dataframe, and then it removes a specific gage at Cape Giradeau to fix conflicts with the related river mile join.

It then imports a river mile GeoJSON into a GeoDataFrame (GDF) and drops matching columns to simplify the following join. The river mile GDF is rounded to 1 decimal place (to account for any floating precision errors), sorted, and reindexed.

The WSE dataframe is then merged with the river mile GDF using the 'mile' field as a key. After that, a column is added for the period and calculated as the maximum value of the time field, converted to an integer, which establishes a time dimension for each point that remains persistent through downstream interpolation.

The GDF is then subset to limit its spatial domain to south of RM 1000 in the vicinity of Cape Giradeau, Missouri and the missing z-values are interpolated using a 1-D linear relationship between river mile and known WSE values.

User-modified variables, such as a bounding box and projection with resolution, are set, and the GDF is formatted as a GeoCube. A POSIX time variable is assigned and expanded as a valid time field. The GeoCube is then clipped to the extent of the Mississippi River and exported as a .NetCDF file.

For the function to work successfully, the script requires the Pyresampled, Geopandas, Pandas, Xarray, MatPlotLib, Cartopy libraries to be properly installed.

*Note that some lines within this script are currently commented out and may require adjusting if using certain versions of software packages.*

In [5]:
def wse_interp(time_query):
    '''This function uses passed period string, called time_query, to create a subset dataframe of Water Surface Elevation (WSE) values from a larger, pre-existing WSE dataframe.Given the subset WSE dataframe, the function then consolidates WSE values with River Mile measurements in a GeoDataFrame via a left/outer merge using river mile as the primary join field. Missing WSE values are interpolated based on a linear relationship between river mile and known WSE values. 

    The GeoDataFrame is then converted into an Xarray, which is further manipulated by dropping NaN values along river mile axis. Dimensions are expanded to incorporate time and user-entered variables such as bounding box, projection information and resolution. Output is exported as a netCDF file and saved to a specified path.'''
        
    # Pass period selection to create subset dataframe 
    period_select = str(time_query)
    wse_slice_df = wse_df.loc[wse_df['time'] == period_select]
    
    # Remove gage at Cape Giradeau to fix conflict with river mile join
    bad_gage = 'rg_CE401278'
    wse_slice_df = wse_slice_df.loc[wse_slice_df['sid'] != bad_gage]
    
    # Import river mile .geojson into gdf and drop matching columns to simplify following join
    url = 'https://raw.githubusercontent.com/hbienn/smartport_wse/main/'
    rm_formatted = f'{url}/mr_rm_banks.geojson'
    rm_gdf = gpd.read_file(rm_formatted, crs='epsg:4326')
    rm_gdf = rm_gdf.drop(columns=['OBJECTID', 'ord', 'sid', 'wse', 'time', 'lat_g', 'lon_g', 'bank']) # add 'bank'
    
    # Round river mile to 1 decimal place to account for any floating precision errors
    rm_gdf = rm_gdf.round({'mile':1})
    rm_gdf = rm_gdf.sort_values('mile')
    rm_gdf = rm_gdf.reindex()
    
    # Merge WSE df with river mile gdf using mile as key
    wse_gdf = rm_gdf.merge(wse_slice_df, how='outer', on='mile')
    
    # Reorder columns for obsessive compulsive reasons
    cols = wse_gdf.columns.tolist()
    cols = ['mile', 'sid', 'z', 'time','lon', 'lat', 'lat_g', 'lon_g', 'geometry']
    wse_gdf = wse_gdf[cols]
    wse_gdf = wse_gdf.sort_values('mile')
    
    # Add column for period and calculate it as max of ['time'] and convert to integer. 
    # Establishes a time dimension for each point that is persistent through the downstream interpolation.
    period = pd.to_datetime(time_query)
    year = period.strftime("%Y")
    day_of_year = period.strftime("%j")
    period = int(round(period.timestamp())*1000000000)
    wse_gdf.insert(4,'period', period)
    
    # Subset gdf to limit spatial domain to south of RM 1000 in the vicinity of Cape Giradeau, MO
    wse_gdf = wse_gdf.loc[wse_gdf['mile'] <= 1000]
    
    # Interpolates missing WSE values based on a linear relationship between river mile and known WSE values.
    #wse_gdf = wse_gdf.dissolve(by='mile', aggfunc='mean')
    wse_gdf = wse_gdf.sort_values('mile')
    wse_gdf['z'] = wse_gdf['z'].interpolate(method='linear', limit_direction = 'both')
    
    # User modified variables
    bounding_box = json.dumps(mapping(box(-91.7,28.9,-89,38.8))) 
    projection = 'EPSG:26915'
    
    # Still issues here with getting make_geocube to recognize time field and assign it correct dtype (datetime64[ns]). 
    # Potentially results from use of a timezone-aware dtype, workaround implemented.
    wse_xr = make_geocube(
                        vector_data = wse_gdf,
                        measurements = ['z',],
                        #datetime_measurements=['period'],
                        output_crs = projection,
                        resolution = (r, r),
                        geom = bounding_box,
                        #interpolate_na_method='linear',
                        rasterize_function=partial(rasterize_points_griddata, method='linear', filter_nan = True)
                        )
    # Expand dimensions and populate with the POSIX time value variable previously assigned 
    period = int(wse_gdf['period'].mean())
    wse_xr = wse_xr.expand_dims('time')
    arr = wse_xr['time'].to_numpy()
    arr[0,] = period
    wse_xr['time'] = arr
    wse_xr['time'] = pd.to_datetime(wse_xr['time'],utc=True)
    period_label = pd.to_datetime(time_query).strftime('%Y%m%dT%H%M')

    # Clip surface to extent of Mississippi River
    url = 'https://raw.githubusercontent.com/hbienn/smartport_wse/main/'
    mr_formatted = f'{url}/generalized_nhdarea_stlouistogulf_utm.geojson'
    mr = gpd.read_file(mr_formatted, crs=projection)
    wse_xr = wse_xr.rio.clip(mr.geometry, mr.crs, drop=True, invert=False)
    
    # Export as .netcdf 
    object_name = f'wse_{resolution}m_{period_label}_y{year}_d{day_of_year}.nc'
    file_for_upload = directory + object_name
    wse_xr.to_netcdf(file_for_upload)
        
    return

for t in wse_period_list:
    if t != wse_period_list[-1]:
        wse_interp(t)
    if t == wse_period_list[-1]:
        break


This code is designed to combine multiple netCDF files into a single daily file for easier analysis. The current implementation iterates through a list (day_list) and combines the 10 netcdfs stored by the matching key value in a dictionary (period_dict). Each output netcdf includes interpolated WSE values for a given day (8) plus the previous (1) and next timestep (1). 

* day_list is a list of integers and associated str entries that represents the days of interest.
* xr.open_mfdataset() is used to open multiple netCDF files at once. In this code, period_dict[str(i)] returns a list of file paths for the ith day in the day_list.
* combine = 'by_coords' specifies that the files should be combined along the coordinate dimensions.
pd.to_datetime() converts the time value at index 2 of the time variable in each netCDF file to a datetime object.
* strftime('%Y%m%d') formats the date as YYYYMMDD string format.
* output_name = f'wse_{r}m_{t}_combine.nc' creates the name for the output file, which includes the resolution r, date t, and file extension .nc.
* ds.to_netcdf(directory + output_name) saves the combined netCDF file to the specified directory with the output_name as the file name.


In [12]:
for i in day_list:
    ds = xr.open_mfdataset(period_dict[str(i)], combine = 'by_coords')
    t = pd.to_datetime(ds['time'].values[2]).strftime('%Y%m%d')
    output_name = f'wse_{r}m_{t}_combine.nc'
    ds.to_netcdf(directory + output_name)

FileNotFoundError: [Errno 2] No such file or directory: b'Z:\\wse\\wse_500m_20221231T2100_y2022_d365.nc'

In [ ]:
import xarray as xr
import pandas as pd
%matplotlib inline

def wse_QC(nc_path):
    ds = xr.open_dataset(nc_path)
    plot = ds.z.where(ds.z!=ds.z.rio.nodata).plot()
    plot.axes.set_xlabel('Easting (m)')
    plot.axes.set_ylabel('Northing (m)')
    title = pd.to_datetime(int(ds['time'])).strftime('%Y%m%dT%H%M')
    plot.axes.set_title(str(title))
    plot.axes.grid(True, linestyle='dotted')
    plot.axes.tick_params()
    plot.axes.ticklabel_format(axis = 'both', style='sci', scilimits=(0,0))
    plot.colorbar.set_label('WSE (NAVD88 m)')
    plot.axes.annotate('PCS: NAD83/UTM Zone 15N', xy=(5,5), xycoords='figure pixels',fontsize=7 )
    plot.figure.set_dpi(150) # 300 is best for export

    return plot

In [ ]:
wse_QC(r'Z:\wse\testing\wse_1000m_20220902_combine.nc')